In [3]:
import os
from openpyxl.workbook import Workbook
from sklearn import clone
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imbens.ensemble import BalancedRandomForestClassifier
from imbens.ensemble import EasyEnsembleClassifier
from imbens.ensemble import KmeansSMOTEBoostClassifier
from instance_selection.operator.metrics import calculate_gmean_mauc
from instance_selection.parameter.parameter import *  # 导入参数的设定
import scipy.io as sio  # 从.mat文件中读取数据集


def data_process(dataset=None):
    mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + dataset.DATASETNAME)  # 加载、划分数据集
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,
                                                        random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    model = MLPClassifier(hidden_layer_sizes=(dataset.HIDDEN_SIZE,), max_iter=dataset.MAX_ITER,
                          random_state=RANDOM_SEED, learning_rate_init=dataset.LEARNING_RATE)
    return x_train, x_test, y_train, y_test, clone(model)


def save_to_excel(data, save_path, filename='avg_results'):
    """
    将列表数据逐行写入Excel文件
    参数:
        data: 二维列表，每个子列表代表一行数据
        filename: 输出的Excel文件名(默认为output.xlsx)
    """
    # 创建一个新的工作簿
    wb = Workbook()
    # 获取活动的工作表
    ws = wb.active
    # 逐行写入数据
    for row in data:
        avg = row[1].tolist()
        avg.insert(0, row[0])
        std = row[2].tolist()
        std.insert(0, row[0])
        ws.append(avg)
        ws.append(std)
    # 创建 Excel 文件完整路径
    file_path = os.path.join(save_path, filename + ".xlsx")
    # 保存Excel文件
    wb.save(file_path)
    print(f"数据已成功写入到 {file_path}")

In [4]:
from imblearn.under_sampling import OneSidedSelection, NearMiss
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import ADASYN
from imbalanced_learning.rbu import RBU
from imblearn.combine import SMOTETomek, SMOTEENN
from imbens.ensemble import BalanceCascadeClassifier, CompatibleBaggingClassifier, RUSBoostClassifier, \
    SMOTEBaggingClassifier
from utils.excel_utils import save_to_excel_2
import numpy as np

DATASETS = [Balance_Scale, Dermatology, Ecoli, Car, Pen_Digits, WallRobot, German, Wine, Nursery, Penbased, USPS,
            Satellite, Page_Blocks, Shuttle, Contraceptive, Automobile, Ovarian]  # 数据集名称（包含对应的参数配置）

if __name__ == "__main__":
    save_path = 'C:/Users/zsc/Desktop/NOS/'
    columns = ['Gmean', 'MAUC']
    datasets_ensembles_results = [[] for _ in range(len(DATASETS))]
    print("*****************算法开始执行：******************")
    for j, dataset in enumerate(DATASETS):
        x_train, x_test, y_train, y_test, model = data_process(dataset=dataset)
        num_run = 40  # 运行次数
        ensembles_results = [[] for _ in range(num_run)]

        for i in range(num_run):
            # clf = EasyEnsembleClassifier(estimator=clone(model), random_state=42 + np.random.randint(0, 1000),
            #                              n_estimators=30) # EasyEnsembleClassifier
            # clf = BalancedRandomForestClassifier(n_estimators=30, random_state=42 + np.random.randint(0, 1000))
            # clf = KmeansSMOTEBoostClassifier(random_state=42 + np.random.randint(0, 1000),
            #                                 n_estimators=30)
            # clf = BalanceCascadeClassifier(estimator=clone(model), random_state=42 + np.random.randint(0, 1000),
            #                               n_estimators=30) 
            clf  = MLPClassifier(hidden_layer_sizes=(dataset.HIDDEN_SIZE,), max_iter=dataset.MAX_ITER,
                          random_state=RANDOM_SEED+ np.random.randint(0, 1000), learning_rate_init=dataset.LEARNING_RATE)
            #resemple = SMOTETomek(random_state=42 + + np.random.randint(0, 1000))
            #resemple = SMOTEENN(random_state=42 + + np.random.randint(0, 1000))
            # resemple = NearMiss()
            # x_res, y_res = resemple.fit_resample(x_train, y_train)
            clf.fit(x_train, y_train)
            y_pred_prob = clf.predict_proba(x_test)  # 默认预测结果是软标签
            gmean, mauc, recall_per_class = calculate_gmean_mauc(y_pred_prob, y_test)

            ensembles_results[i] = [gmean, mauc]
            print(f"第{i + 1}次执行：Gmean：{gmean}，mAUC：{mauc}")
        save_to_excel_2(save_path + dataset.DATASETNAME.split('.')[0] + '/', dataset.DATASETNAME.split('.')[0], columns,
                        ensembles_results)
        ensembles_result_mean = np.mean(ensembles_results, axis=0)
        # 计算ensembles_resultsz中每一列的标准差
        ensembles_result_std = np.std(ensembles_results, axis=0)
        print(f'集成分类结果（平均值）：{ensembles_result_mean}')
        print(f'集成分类结果（标准差）：{ensembles_result_std}')
        datasets_ensembles_results[j] = [dataset.DATASETNAME.split('.')[0], ensembles_result_mean, ensembles_result_std]
    print("*****************算法执行结束！******************")
    # 写入到Excel     
    save_to_excel(datasets_ensembles_results, save_path)       

*****************算法开始执行：******************
第1次执行：Gmean：0.958173，mAUC：0.991059
第2次执行：Gmean：0.958354，mAUC：0.988783
第3次执行：Gmean：0.923704，mAUC：0.989085
第4次执行：Gmean：0.913867，mAUC：0.991798
第5次执行：Gmean：0.869848，mAUC：0.983033
第6次执行：Gmean：0.962142，mAUC：0.997419
第7次执行：Gmean：0.863422，mAUC：0.989033
第8次执行：Gmean：0.980603，mAUC：0.997818
第9次执行：Gmean：0.988371，mAUC：0.997933
第10次执行：Gmean：0.984525，mAUC：0.998041
第11次执行：Gmean：0.980649，mAUC：0.994339
第12次执行：Gmean：0.943034，mAUC：0.994878
第13次执行：Gmean：0.973506，mAUC：0.998969
第14次执行：Gmean：0.923523，mAUC：0.995132
第15次执行：Gmean：0.722514，mAUC：0.981933
第16次执行：Gmean：0.913952，mAUC：0.992875
第17次执行：Gmean：0.927355，mAUC：0.982972
第18次执行：Gmean：0.992187，mAUC：0.997523
第19次执行：Gmean：0.988371，mAUC：0.999355
第20次执行：Gmean：0.873503，mAUC：0.980874
第21次执行：Gmean：0.984433，mAUC：0.992579
第22次执行：Gmean：0.962142，mAUC：0.995432
第23次执行：Gmean：0.996154，mAUC：0.999745
第24次执行：Gmean：0.962142，mAUC：0.996298
第25次执行：Gmean：0.969718，mAUC：0.999614
第26次执行：Gmean：0.996154，mAUC：0.999745
第27次执行：Gmean：0.894734，mAUC：0.9